# Lesson 03 Assignment

## Background

    A superstore wants to identify customer groupings that visit their stores based on customer transaction (sales) data. These groupings are used for targeted promotions. You are asked to perform customer segmentation on the provided data.

## Instructions

    Using the Superstore Transaction dataset, create a new notebook and perform each of the following tasks and answer the related questions:

    (1) Read dataset
    (2) Calculate the Recency, Frequency, Monitory for each customer
    (3) Using K-means algorithm, perform customer segmentation
    (4) Describe your results

In [ ]:
# Import packages

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import OrderedDict
import datetime as dt
import matplotlib.pyplot as plt

#Plot styling

import seaborn as sns; sns.set()  # for plot styling
%matplotlib inline

### (1) Read Data

In [ ]:
# Reading url

data = pd.read_csv("/Users/matt.denko/Downloads/SuperstoreTransaction.csv") 
data.columns = ["Row ID",
"Order ID",
"Order Date",
"Ship Date",
"Ship Mode",
"Customer ID",
"Customer Name",
"Segment",
"Country",
"City",
"State",
"Postal Code",
"Region",
"Product ID",
"Category",
"Sub-Category",
"Product Name",
"Sales",
"Quantity",
"Discount",
"Profit"]
print(data.columns)
data.describe()
data.head()

In [ ]:
#Removing cases with missing data

data = data.replace(to_replace= "?", value=float("NaN"))
data_null = data.isnull().sum()
print(data_null)
print("There are 0 columns with missing data")

### (2) Calculate the Recency, Frequency, Monitory for each customer

#### Convert Timestamp to Datetime Object

In [ ]:
# Order Date

data["Order Date"] = pd.to_datetime(data["Order Date"], format='%m/%d/%Y')
data.head()
print("Minimal Order Date=%s, Maximal Order Date=%s"%(min(data["Order Date"]).strftime("%Y-%m-%d %H:%M"), \
                                          max(data["Order Date"]).strftime("%Y-%m-%d %H:%M")))

# Ship Date

data["Ship Date"] = pd.to_datetime(data["Ship Date"], format='%m/%d/%Y')
data.head()
print("Minimal Ship Date=%s, Maximal Ship Date=%s"%(min(data["Ship Date"]).strftime("%Y-%m-%d %H:%M"), \
                                          max(data["Ship Date"]).strftime("%Y-%m-%d %H:%M")))

In [ ]:
# Recency, Frequency, Monitory Calculation

Start_Date_Obj = dt.datetime.strptime("1/1/2014", "%m/%d/%Y")
End_Date_Obj = dt.datetime.strptime("1/10/2018", "%m/%d/%Y")
Time_Window = 60 #days. Only consider customers who have activities within the recent 60 days
FM_Window = 7 #days for frequency and monetary

check_point_date = Start_Date_Obj
UserID = []
Checkpoint = []
Recency = []
Frequency = []
Monetary_Value = []
Monetary_Quantity = []
while check_point_date <= End_Date_Obj:
    window_start = check_point_date - dt.timedelta(days = Time_Window)
    mask = (data["Order Date"] >= window_start) & (data["Order Date"] < check_point_date)
    # Get the data in [checkpoint-60days, checkpoint]
    data_checkpoint = data.loc[mask]
    # Get the ids of users who have activities in [checkpoint-60days, checkpoint]
    unique_users = list(set(data_checkpoint["Customer ID"]))
    print("There are %d unique users."%(len(unique_users)))
    FM_Window_Start = check_point_date - dt.timedelta(days = FM_Window)
    for user in unique_users:
        UserID.append(user)
        Checkpoint.append(check_point_date)
        mask = data_checkpoint["Customer ID"] == user
        data_checkpoint_user = data_checkpoint.loc[mask]
        delta = check_point_date - max(data_checkpoint_user["Order Date"])
        recency = delta.days #Recency, days between checkpoint and last transaction time
        mask = data_checkpoint_user["Order Date"] >= FM_Window_Start
        data_checkpoint_user_fm = data_checkpoint_user.loc[mask]
        frequency = data_checkpoint_user_fm.shape[0]
        value = np.sum(data_checkpoint_user_fm.iloc[:, 8]) #monetary values
        quantity = np.sum(data_checkpoint_user_fm.iloc[:, 7])#monetary quantity
        Recency.append(recency)
        Frequency.append(frequency)
        Monetary_Value.append(value)
        Monetary_Quantity.append(quantity)
    check_point_date = check_point_date + dt.timedelta(days = 1)
# Consolidate all columns into a signle data frame
RFM_Dict = OrderedDict([ ('Customer ID', UserID),
          ('Checkpoint', Checkpoint),
          ('Recency',  Recency),
          ('Frequency', Frequency),
          ('Value', Monetary_Value),
          ('Quantity', Monetary_Quantity)] )
RFM_Frame = pd.DataFrame.from_dict(RFM_Dict)
RFM_Frame.head()

### (3) Using K-means algorithm, perform customer segmentation

In [ ]:
# Subsetting Data for columns to use for k-means

X = pd.DataFrame()
X.loc[:,"Sales"] = data.loc[:,"Sales"]
X.loc[:,"Profit"] = data.loc[:,"Profit"]
print(X)

In [ ]:
# Using the elbow method to find the optimum number of clusters

from sklearn.cluster import KMeans
wcss = []
for i in range(1,10):
    km=KMeans(n_clusters=i,init='k-means++', max_iter=300, n_init=10, random_state=0)
    km.fit(X)
    wcss.append(km.inertia_)
plt.plot(range(1,10),wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('wcss')
plt.show()

In [ ]:
# Fitting kmeans to the dataset with k=3

X = np.array(X)
km4=KMeans(n_clusters=3,init='k-means++', max_iter=10, n_init=10, random_state=0)
y_means = km4.fit_predict(X)
print(y_means)

# Visualizing the clusters for k=3

plt.scatter(X[y_means==0,0],X[y_means==0,1],s=50, c='purple',label='Cluster1')
plt.scatter(X[y_means==1,0],X[y_means==1,1],s=50, c='blue',label='Cluster2')
plt.scatter(X[y_means==2,0],X[y_means==2,1],s=50, c='green',label='Cluster3')
plt.scatter(km4.cluster_centers_[:,0], km4.cluster_centers_[:,1],s=200,marker='s', c='red', alpha=0.7, label='Centroids')
plt.title('Customer segments')
plt.xlabel('Annual sales of customer')
plt.ylabel('Annual profit from customer')
plt.legend()
plt.show()

### (4) Describe your results

#### Comments:

    In the Recency, Frequency, Monetary calculation the slices with the highest Recency and Frequency scores were two customer ids in United States - Consumer.
    
    In the k-means customer segmentation based off the elbow graph I chose 3 clusters. Cluster 1 and 2 appear to have some overlap while cluster 3 is very spread out.